In [66]:
import globus_sdk
import datetime
import time
import pytz
import pandas as pd
import calendar
import numpy as np

In [12]:
CLIENT_ID = 'ac34a1cf-7412-4f43-a693-935d7d09fd85'

client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))

# this is to work on Python2 and Python3 -- you can just use raw_input() or
# input() for your specific version
get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input(
    'Please enter the code you get after login here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server['auth.globus.org']
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']

# most specifically, you want these tokens as strings
AUTH_TOKEN = globus_auth_data['access_token']
TRANSFER_TOKEN = globus_transfer_data['access_token']

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=ac34a1cf-7412-4f43-a693-935d7d09fd85&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=8j1Ep5hQ2M0C4HlPzmNrmgFHQeMTCZIkT8gRIorinbg&code_challenge_method=S256&access_type=online
Please enter the code you get after login here: CXd6BYUVGHTZFv4KMNPzghbg0mGZgz


In [13]:
# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# more advanced scenarios, other types of GlobusAuthorizers give us
# expressive power
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

# high level interface; provides iterators for list responses
print("My Endpoints:")
for ep in tc.endpoint_search(filter_scope="my-endpoints"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))

My Endpoints:
[18c03e00-8157-11e8-94e4-0a6d4e044368] nersc_transfer


In [14]:
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()

print('Please go to this URL and login: {0}'
      .format(client.oauth2_get_authorize_url()))

get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input('Please enter the code here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=ac34a1cf-7412-4f43-a693-935d7d09fd85&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=K_FHEoEK_e_xoeL_AgJBIRl1lJyy70PdnRe5Erbchg0&code_challenge_method=S256&access_type=online
Please enter the code here: T2mSPlmMPG5ymFl4qXKHIWYb61DtGi


In [15]:
str(token_response.by_resource_server)

"{'auth.globus.org': {'scope': 'profile openid email', 'access_token': 'AgyNv0pyWaDYQEKwzqvopy5g8ymq01nGxXyqEe3YE0GmwNwDwBFVC7vYYx8gpnD7Jw8MaNDvlgyXmQSONY2X6hP5WvUpOXdsqwNm', 'refresh_token': None, 'token_type': 'Bearer', 'expires_at_seconds': 1536092330, 'resource_server': 'auth.globus.org'}, 'transfer.api.globus.org': {'scope': 'urn:globus:auth:scope:transfer.api.globus.org:all', 'access_token': 'AgpoP3WgjO4y3BBxV7jok246wGGlKn3yobbe60EBpr51OeXlglivCXGGpOvz2QveqNqPDdp4kJqQBYFXvnp26Im3V2', 'refresh_token': None, 'token_type': 'Bearer', 'expires_at_seconds': 1536092330, 'resource_server': 'transfer.api.globus.org'}}"

In [16]:
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow(refresh_tokens=True)

print('Please go to this URL and login: {0}'
      .format(client.oauth2_get_authorize_url()))

get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input('Please enter the code here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=ac34a1cf-7412-4f43-a693-935d7d09fd85&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=-aOAPlLoQOjbdpjzapjN5VawTZB8J8U5_rBsdbFI5gw&code_challenge_method=S256&access_type=offline
Please enter the code here: TGXouT9wYJWUfHntSTWnGLBZu4EGfP


In [17]:
str(token_response.by_resource_server)

"{'auth.globus.org': {'scope': 'profile openid email', 'access_token': 'Ag8qXJNEWEv5WKrqbyB6zDnq3lepgkBykwPGndXldQl5gE20vwT8Cjqx6zn1grqjawzbeneev9jmYKI7VmK9nFB9v6S3pMbuKMaK', 'refresh_token': 'AgJq7qNgby3Xvq4P9QkyQqjxEo5XX66qVyblv6WgnOwNp2lgneCGU5yV7WMMDMGGWPMVo4nYmYr57PGM78BqvyO6ynj1q', 'token_type': 'Bearer', 'expires_at_seconds': 1536092347, 'resource_server': 'auth.globus.org'}, 'transfer.api.globus.org': {'scope': 'urn:globus:auth:scope:transfer.api.globus.org:all', 'access_token': 'AgGBmk3Mpej307Kwm6n3jpXb16JxlbQNqlmb0zXO748O2g2lP7U7CG2wqb7BmjKoG61ggy9nKQVyW9hQdnMV2c5wQr', 'refresh_token': 'AgnBPxj49g836qyKXyXx031B9kBDkN7pN8NYE5eOlwPpd5rWMmudUnDJvoJ2ekPqnw4W239nqeXxqNoPbP0YVOEelmeXz', 'token_type': 'Bearer', 'expires_at_seconds': 1536092347, 'resource_server': 'transfer.api.globus.org'}}"

In [18]:
# let's get stuff for the Globus Transfer service
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']
# the refresh token and access token, often abbr. as RT and AT
transfer_rt = globus_transfer_data['refresh_token']
transfer_at = globus_transfer_data['access_token']
expires_at_s = globus_transfer_data['expires_at_seconds']

# Now we've got the data we need, but what do we do?
# That "GlobusAuthorizer" from before is about to come to the rescue

authorizer = globus_sdk.RefreshTokenAuthorizer(
    transfer_rt, client, access_token=transfer_at, expires_at=expires_at_s)

# and try using `tc` to make TransferClient calls. Everything should just
# work -- for days and days, months and months, even years
tc = globus_sdk.TransferClient(authorizer=authorizer)


In [58]:
from globus_sdk import TransferClient

# you must have transfer_token in your config for this to work
#tc = TransferClient()

print("My Last 25 Tasks:")
# `filter` to get Delete Tasks (default is just Transfer Tasks)
for task in tc.task_list(num_results=1, filter="type:TRANSFER,DELETE"):
    print(task["task_id"], task["type"], task["status"], task['effective_bytes_per_second'])

My Last 25 Tasks:
3484ea16-aef8-11e8-823c-0a3b7ca8ce66 TRANSFER SUCCEEDED 5047408402


In [15]:
tdata = globus_sdk.TransferData(tc, "9d6d994a-6d04-11e5-ba46-22000b92c6ec",
                                 "e09e65f5-6d04-11e5-ba46-22000b92c6ec",
                                 label="nogrowcheckplease")
                                 #sync_level="checksum")
#tdata.add_item("/source/path/dir/", "/dest/path/dir/", recursive=True)
tdata.add_item("/global/cscratch1/sd/yuanlai/gf_test/new_test_files_all_ost/", "/projects/AMASE/lyl/new_test_files_all_ost/", recursive=True)
#tdata.add_item("/global/homes/y/yuanlai/workspace/DTNSpeedTest/mpi_fread_test/s_fname_list.txt", "/dev/null")
#tdata.add_item("/global/homes/y/yuanlai/workspace/DTNSpeedTest/mpi_fread_test/s_fname_list.txt", "/dev/null")
transfer_result = tc.submit_transfer(tdata)
print("task_id =", transfer_result["task_id"])

task_id = ae6f08a0-aedc-11e8-823c-0a3b7ca8ce66


In [21]:
nersc_id = "9d6d994a-6d04-11e5-ba46-22000b92c6ec"
alcf_id = "e09e65f5-6d04-11e5-ba46-22000b92c6ec"
from_big_dir = "/global/cscratch1/sd/yuanlai/gf_test/read_test_files/"
to_big_dir = "/projects/AMASE/lyl/read_test_files/"
from_small_dir = "/global/cscratch1/sd/yuanlai/gf_test/new_test_files_all_ost/"
to_small_dir = "/projects/AMASE/lyl/new_test_files_all_ost/"

In [30]:
now = datetime.datetime.utcnow()
ddl = now + datetime.timedelta(minutes=10)
tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id, label="nogrowcheckplease", deadline=str(ddl))
for i in range(100):
    tdata.add_item("/dev/zero", "/dev/null")
transfer_result = tc.submit_transfer(tdata)
print("task_id =", transfer_result["task_id"])

task_id = 3a5534b2-aef0-11e8-823c-0a3b7ca8ce66


In [ ]:
(pd.to_datetime('2018-09-02T22:01:40.000Z') - pd.datetime.utcfromtimestamp(0)).total_seconds().__int__()

In [52]:
calendar.timegm(now.utctimetuple())

1535921413

In [ ]:
big_transfer_speed = []
small_transfer_speed = []
for i in range(10):
    tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id, label="raj-cc-96")
    tdata.add_item(from_big_dir, to_big_dir, recursive=True)
    transfer_result = tc.submit_transfer(tdata)
    while not tc.task_wait(transfer_result["task_id"], timeout=10):
        pass
    big_transfer_speed.append(tc.get_task(transfer_result["task_id"])['effective_bytes_per_second'])
    
    tdata = globus_sdk.TransferData(tc, nersc_id, alcf_id, label="raj-cc-96")
    tdata.add_item(from_small_dir, to_small_dir, recursive=True)
    transfer_result = tc.submit_transfer(tdata)
    while not tc.task_wait(transfer_result["task_id"], timeout=10):
        pass
    small_transfer_speed.append(tc.get_task(transfer_result["task_id"])['effective_bytes_per_second'])
print(big_transfer_speed)
print(small_transfer_speed)
np.savez("data/bigVSsmall_10_speeds.npz", name1 = big_transfer_speed, name2 = small_transfer_speed)


In [71]:
np.load("data/bigVSsmall_10_speeds.npz")["name1"]/np.load("data/bigVSsmall_10_speeds.npz")["name2"]

array([2.00492703, 1.73845353, 2.02000409, 2.09034961, 2.35632986,
       1.99736833, 1.94575394, 2.09725914, 2.18795697, 2.11784687])